In [124]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt 

from torch import nn
from torch import optim
from torchvision import transforms, datasets
import data_setup, engine, model_builder
from PIL import Image, ImageDraw
from bs4 import BeautifulSoup
import dlib
import face_recognition

In [125]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [126]:
# Get data
import os
import shutil
import requests
import zipfile

from pathlib import Path

# Setup path to data folder
image_path = Path("./AmericanCelebrities/")

# if the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

# Setup Dirs
train_dir = image_path / "train"
valid_dir = image_path / "valid"
test_dir = image_path / "test"
train_dir.mkdir(parents=True, exist_ok=True)
valid_dir.mkdir(parents=True, exist_ok=True)
test_dir.mkdir(parents=True, exist_ok=True)

AmericanCelebrities directory exists.


In [127]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm.auto import tqdm
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import urllib
import time

In [128]:
chrome_path = './chromedriver.exe'

In [129]:
chrome_options = Options()
chrome_options.add_argument('--headless') # 창 없는 모드
chrome_options.add_argument('lang=ko_KR') # 한국어
chrome_options.add_argument('window-size=1920x1080')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(chrome_path, options=chrome_options)

C:\Users\young\AppData\Local\Temp\ipykernel_16028\201190133.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(chrome_path, options=chrome_options)


In [131]:
from requests import get  

def download(url, file_name = None):
    if not file_name:
        file_name = url.split('/')[-1]

    with open(file_name, "wb") as file:   
        response = get(url)               
        file.write(response.content)      

url = "https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml"
download(url)

In [ ]:
import torchvision.transforms as T

def get_prediction(img_path, threshold):
    img = Image.open(img_path)
    transform = T.Compose([T.ToTensor(),
                           T.Normalize(mean=[0.485, 0.456, 0.406],
                                       std=[0.229, 0.224, 0.225])
    ])
    pred = model(transform(img.convert('RGB')).unsqueeze(dim=0).to(device))
    pred_score = list(pred[0]['scores'].cpu().detach().numpy())
    if len(pred_score) == 0:
        return None, None, None
    pred_t = [pred_score.index(x) for x in pred_score if x>threshold][-1]    # [-1]은 x가 threshold 보다 index 증 맨 마지막 index 
    masks = (pred[0]['masks']>0.5).squeeze().detach().cpu().numpy()
    pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].cpu().detach().numpy())]
    pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].cpu().detach().numpy())]
    masks = masks[:pred_t+1]
    pred_boxes = pred_boxes[:pred_t+1]
    #print(pred_boxes)
    pred_class = pred_class[:pred_t+1]
    return masks, pred_boxes, pred_class

In [130]:
from PIL import Image

def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    # TODO: Process a PIL image for use in a PyTorch model
    # Resize the images where the shortest side is 256 pixels, keeping the aspect ratio
    #pil_image = Image.convert('RGB')   
    width, height = image.size
    aspect_ratio = width / height
    if aspect_ratio > 1:
        pil_image = image.resize((round(aspect_ratio * 256), 256))
    else:
        pil_image = image.resize((256, round(256 / aspect_ratio)))
        
    # Crop out the center 224x224 portion of the image
    width, height = pil_image.size
    new_width = 224
    new_height = 224
    left = (width - new_width)/2
    top = (height - new_height)/2
    right = (width + new_width)/2
    bottom = (height + new_height)/2
    pil_image = pil_image.crop((round(left), round(top), round(right), round(bottom)))
    
    return pil_image

In [131]:
import face_recognition
#This is my choice.
# 크롬 브라우저 내부 대기
browser.implicitly_wait(5)
SCROLL_PAUSE_TIME = 5

celebrity_list = ['Beyoncé', 'Hillary Clinton', 'Johnny Depp', 
          'Tom Cruise', 'Barack Obama', 'Will Smith', 
          'Lady Gaga', 'Michelle Obama', 'Whoopi Goldberg', 
          'Britney Spears']
celebrity_list.sort()
for name in celebrity_list:
    namer = name.replace(' ', '+')
    namee = name.replace(' ', '')
    a = b = c = 0
    k1 = train_dir/namee
    k1.mkdir(parents=True, exist_ok=True)
    k2 = valid_dir/namee
    k2.mkdir(parents=True, exist_ok=True)
    k3 = test_dir/namee
    k3.mkdir(parents=True, exist_ok=True)
    url = [f"https://search.naver.com/search.naver?where=image&sm=tab_jum&query={namer}",
               f"https://www.google.co.kr/search?q={namer}&tbm=isch&ved=2ahUKEwjPoaDL4of-AhV1tFYBHbcKD58Q2-cCegQIABAA&oq=daniel+&gs_lcp=CgNpbWcQARgAMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDoECCMQJzoICAAQgAQQsQM6CwgAEIAEELEDEIMBUMwQWMoaYIEwaABwAHgAgAFSiAH0BJIBATiYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=7agnZI_3NvXo2roPt5W8-Ak&bih=827&biw=1458",
               f"https://www.bing.com/images/search?q={namer}&form=HDRSC3&first=1",
               f"https://search.daum.net/search?w=img&nil_search=btn&DA=NTB&enc=utf8&q={namer}"]
    for i in range(len(url)):
    
        browser.get(url[i])

        browser.implicitly_wait(10)

        last_height = browser.execute_script("return document.body.scrollHeight")
        # 페이지 가장 하단의 창 높이
        src = []
        # scrape images of a celebrity by using limitless scroll and selenium (XPATH)
        while True:
            if i == 0:
                imgResults = browser.find_elements(By.XPATH, "//*[@id='main_pack']//a[contains(@class, 'link_thumb')]/img[contains(@class, '_listImage')]")
                for img in imgResults:
                    if img.get_attribute('src') not in src:
                        src.append(img.get_attribute('src'))
            elif i == 1:
                imgResults = browser.find_elements(By.XPATH, "//*[@id='islrg']//div[contains(@class, 'islir')]/img[contains(@class, 'Q4LuWd')]")
                for img in imgResults:
                    if img.get_attribute('src') not in src:
                        src.append(img.get_attribute('src'))
            elif i == 2:
                imgResults = browser.find_elements(By.XPATH, "//*[@id='mmComponent_images_2']/ul//img[contains(@class, 'mimg')]")
                for img in imgResults:
                    if img.get_attribute('src') not in src:
                        src.append(img.get_attribute('src'))
            else:
                imgResults = browser.find_elements(By.XPATH, "//*[@id='imgList']/div//img[@class='thumb_img']")
                for img in imgResults:
                    if img.get_attribute('src') not in src:
                        src.append(img.get_attribute('src'))

            # Scroll down to bottom
            browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = browser.execute_script("return document.body.scrollHeight")

            if new_height == last_height:
                break

            last_height = new_height        
        # save training images
        for r in range(len(src)):
            position = f"{str(k1)}/{a+1}.jpg"
            urllib.request.urlretrieve(str(src[r]), position)
            image = face_recognition.load_image_file(position)
            face_locations = face_recognition.face_locations(image)
            if len(face_locations) == 1:
                top, right, bottom, left = face_locations[0]
                face_image = image[top:bottom, left:right]
                pil_image = Image.fromarray(face_image)
                pil_image = process_image(pil_image)
                pil_image.save(position)
                a += 1
                if a >= 20 * (i+1):
                    src = src[r+1:]
                    break
            else:
                os.remove(position)
        # save validation images
        for r in range(len(src)):
            position = f"{str(k2)}/{b+1}.jpg"
            urllib.request.urlretrieve(str(src[r]), position)
            image = face_recognition.load_image_file(position)
            face_locations = face_recognition.face_locations(image)
            if len(face_locations) == 1:
                top, right, bottom, left = face_locations[0]
                face_image = image[top:bottom, left:right]
                pil_image = Image.fromarray(face_image)
                pil_image = process_image(pil_image)
                pil_image.save(position)
                b += 1
                if b >= 2 * (i+1):
                    src = src[r+1:]
                    break
            else:
                os.remove(position)
        # save test images    
        for r in range(len(src)):
            position = f"{str(k3)}/{c+1}.jpg"
            urllib.request.urlretrieve(str(src[r]), position)
            image = face_recognition.load_image_file(position)
            face_locations = face_recognition.face_locations(image)
            if len(face_locations) == 1:
                top, right, bottom, left = face_locations[0]
                face_image = image[top:bottom, left:right]
                pil_image = Image.fromarray(face_image)
                pil_image = process_image(pil_image)
                pil_image.save(position)
                c += 1
                if c >= 3 * (i+1):
                    break
            else:
                os.remove(position)

        # 대기
        browser.implicitly_wait(5)

        # 스크롤 맨 위로 올리기
        browser.execute_script('window.scrollTo(0,0)')
browser.close()